In [2]:
import datetime#import datetime module convert data to datetime64(ns)
import numpy as np
import pandas as pd#import pandas module(Better than csv module!)
import os
import glob
import sklearn as sk
from sklearn.model_selection import train_test_split
from statistics import mean

def resha(x):
    nptrain=np.array(x)
    nptrain=np.reshape(nptrain,(nptrain.shape[0]*nptrain.shape[1], nptrain.shape[2]))
    return nptrain
    
def save_np(x,y):
    train_x,x_test,train_y,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
    stock_name=path[12:21]
    scaler = sk.preprocessing.StandardScaler().fit(resha(train_x))  #  標準化後的數據 
    train_x=sk.preprocessing.scale(resha(train_x),0) # normalize each feature
    
    Npdata=train_x
    np.save(os.path.join('./StockData/TrainingData/','NormtrainingX_'+stock_name),Npdata)
    print(path[12:21]," trainX  ",Npdata.shape)
    #print(Npdata)
    Npdata=scaler.transform(resha(x_test))# normalize x_test with scale of train_x
    np.save(os.path.join('./StockData/TrainingData/','NormtestingX_'+stock_name),Npdata)
    print(path[12:21]," testX  ",Npdata.shape)
    #print(Npdata)
    
    Npdata=np.array(train_y)
    np.save(os.path.join('./StockData/TrainingData/','trainingY_'+stock_name),Npdata)
    print(path[12:21]," trainY  ",Npdata.shape)
    #print(Npdata)
    
    Npdata=np.array(y_test)
    np.save(os.path.join('./StockData/TrainingData/','testingY_'+stock_name),Npdata)
    print(path[12:21]," testY  ",Npdata.shape)
    #print(Npdata)
    

def add_MA5(data):
    close=data.loc[ : ,'close'].values.tolist()
    ma5=[]
    for i in range(0,len(close)):
        if i>=4:
            ma5.append(sum(data.loc[ i-4 : i ,'close'].values.tolist())/5)
        else:
            ma5.append(0)
    return ma5

def add_MA20(data):
    close=data.loc[ : ,'close'].values.tolist()
    ma20=[]
    for i in range(0,len(close)):
        if i>=20:
            ma20.append(sum(data.loc[ i-20 : i ,'close'].values.tolist())/20)
        else:
            ma20.append(0)
    return ma20

def add_MA60(data):
    close=data.loc[ : ,'close'].values.tolist()
    ma60=[]
    for i in range(0,len(close)):
        if i>=60:
            ma60.append(sum(data.loc[ i-60 : i ,'close'].values.tolist())/60)
        else:
            ma60.append(0)
    return ma60

def add_rsv(data): # rsv (今天收盤-最近9天的最低價)/(最近9天的最高價-最近9天的最低價)
    rsv=[]
    close=data.loc[ : ,'close'].values.tolist() 
    for i in range(0,len(close)):
        if i>=8:
            low=min(data.loc[ i-8 : i ,'low'].values.tolist())
            high=max(data.loc[ i-8 : i ,'high'].values.tolist()) 
            rsv.append(((close[i]-low)/(high-low))*100)
        else:
            rsv.append(0) 
    return rsv

def add_k(data):  # k (2/3昨日K 加 1/3 今日rsv)
    k=[] 
    rsv=data.loc[ : ,'rsv'].values.tolist() 
    for i in range(0,len(rsv)):
        if i>=1:
            k.append(((2/3)*k[i-1])+((1/3)*rsv[i]))
        else:
            k.append(rsv[0])       
    return k

def add_d(data): # d (2/3昨日d 加 1/3 今日k)
    d=[]
    k=data.loc[ : ,'k'].values.tolist() 
    for i in range(0,len(k)):
        if i>=1:
            d.append(((2/3)*d[i-1])+((1/3)*k[i]))
        else:
            d.append(k[0])            
    return d

def generate_train(close_type,feature,data):
    train_x=[]
    train_y=[]
    for tmp in data:
        # print(tmp[1])
        i=tmp[1].dropna()
        if len(i)==5: #Decide the way seperate the stock data
            train_x.append(i.loc[ : ,feature].values.tolist())
            #print(train_x)
            Open=i.loc[ : ,'open'].values.tolist()[0]
            Close=i.loc[ : ,close_type].values.tolist()[len(i.loc[ : ,close_type].values.tolist())-1]
            train_y.append(float("%.2f" %((Close-Open))))
        else:
            continue
    #print(train_y)
    #exit()
    save_np(train_x,train_y)

feature=input("Enter the needed features seperated with space: ").split(" ")
ctype=input("Enter the close type: ")
resam=input("Enter the desired  data span , d  for day  w  for week  m  for month: ")

for path in glob.glob(r'./StockData/stock0050.csv'):
    csv_data=pd.DataFrame(pd.read_csv(path))
    csv_data['date']=pd.to_datetime(csv_data['date'])
    csv_data=csv_data.drop([0],axis=0)#drop 第一天 因為stockdata 有16年跳到17年的問題
    
    csv_data['rsv']=add_rsv(csv_data)
    csv_data['k']=add_k(csv_data)
    csv_data['d']=add_d(csv_data)
    csv_data['ma5']=add_MA5(csv_data)
    csv_data['ma20']=add_MA20(csv_data)
    csv_data['ma60']=add_MA60(csv_data)
    
    print(csv_data)
    csv_data=csv_data.set_index('date').resample(resam)#group
    generate_train(ctype,feature,csv_data)
   

Enter the needed features seperated with space: close open rsv k d volume ma20 ma60
Enter the close type: close
Enter the desired  data span , d  for day  w  for week  m  for month: w
           date  adj_close  close   high    low   open   volume         rsv  \
1    2004-02-12      48.54  48.59  49.26  48.40  48.50  8114000    0.000000   
2    2004-02-13      49.41  49.46  49.47  48.60  48.61  6138000    0.000000   
3    2004-02-16      49.17  49.20  49.46  49.04  49.46  4154000    0.000000   
4    2004-02-17      49.40  49.50  49.50  49.16  49.21  4645000    0.000000   
5    2004-02-18      49.53  49.53  49.70  49.32  49.55  4018000    0.000000   
...         ...        ...    ...    ...    ...    ...      ...         ...   
3912 2019-12-05      92.85  92.90  93.00  92.20  92.35  3441050   78.947368   
3913 2019-12-06      92.95  93.05  93.50  92.75  93.30  3394433   86.842105   
3914 2019-12-09      93.60  93.65  93.80  93.20  93.20  2912194  107.142857   
3915 2019-12-10      93.00

No handles with labels found to put in legend.


stock0050  trainX   (2515, 8)


<Figure size 640x480 with 1 Axes>

stock0050  testX   (840, 8)
stock0050  trainY   (503,)
stock0050  testY   (168,)
